In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

% matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
date_col = ['issue_d','last_pymnt_d','earliest_cr_line','last_credit_pull_d']
def lookup(s):
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

#print(os.listdir("../input"))
data = pd.read_csv("../input/loan.csv")
for i in date_col:
    data[i] = lookup(data[i])

    #print(data.iloc[1])
data_null = data.isna().sum()
#print(data_null)
#print(data_null['id'])
check_size = round(data.shape[0]/100)
keys =[]
for key, values in data_null.items():
    if values > check_size:
        keys.append(key)
data.drop(keys, axis=1,inplace=True)
data = data.dropna(how='any',axis=0)
data = data.reset_index(drop=True)
print(data.shape)
data = data.drop(columns='zip_code')



In [ ]:
data['issue_days'] = np.nan
data['credit_days'] = np.nan
def diffrence_days(start,end):
    return (start-end).days

data['issue_days'] = data.apply(lambda x: diffrence_days(x.last_pymnt_d, x.issue_d), axis=1)
data['credit_days'] = data.apply(lambda x: diffrence_days(x.last_credit_pull_d, x.earliest_cr_line), axis=1)
data = data.drop(columns=date_col)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,5))
loan_amount = data["loan_amnt"].values
funded_amount = data["funded_amnt"].values
investor_funds = data["funded_amnt_inv"].values


sns.distplot(loan_amount, ax=ax[0])
ax[0].set_title("Loan Applied by the Borrower", fontsize=14)
sns.distplot(funded_amount, ax=ax[1])
ax[1].set_title("Amount Funded by the Lender", fontsize=14)
sns.distplot(investor_funds, ax=ax[2])
ax[2].set_title("Total committed by Investors", fontsize=14)

In [ ]:
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']



data['region'] = np.nan

def finding_regions(state):
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    


data['region'] = data['addr_state'].apply(finding_regions)
data = data.drop(columns='addr_state')
data = data.drop(columns='sub_grade')

In [ ]:
data = data.reindex(list([a for a in data.columns if a != 'loan_status'] + ['loan_status']), axis=1)
y = data.iloc[:,-1]
X = data.iloc[:,0:len(data.columns)-1]

In [ ]:
cols = data.columns
num_cols = data._get_numeric_data().columns
date_cols = data.select_dtypes(include=[np.datetime64]).columns.tolist()
cat_data = list(set(cols) - set(num_cols) - set(date_cols))


In [ ]:
label = []
hot = []
remove = []
for i in cat_data:
    cnt = data[i].nunique()
    if cnt == 2 or i == 'loan_status':
        label.append(i)
    elif cnt ==1:
        remove.append(i)
    else:
        hot.append(i)
data = data.drop(columns=remove)
print(label,hot)

In [ ]:
data = pd.get_dummies(data, columns=hot, drop_first=True)
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns 

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = preprocessing.LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = preprocessing.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

data = MultiColumnLabelEncoder(columns = label).fit_transform(data)



In [ ]:
corr = data.corr()
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False

selected_columns = data.columns[columns]

data = data[selected_columns]

In [ ]:
rem = list(set(data.columns) - set(selected_columns))
print(rem)
sns.heatmap(corr)

In [ ]:
using_tree = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'inq_last_6mths', 'open_acc', 'revol_bal', 'revol_util', 'total_acc','loan_status', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'last_pymnt_amnt', 'issue_days', 'credit_days', 'verification_status_Source Verified', 'verification_status_Verified']
data1 = copy.deepcopy(data[using_tree])

In [ ]:
data1 = data1.reindex(list([a for a in data1.columns if a != 'loan_status'] + ['loan_status']), axis=1)
y = data1.iloc[:,-1]
X = data1.iloc[:,0:len(data1.columns)-1]

In [ ]:

#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


from sklearn.model_selection import StratifiedShuffleSplit

stratified = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_set, test_set in stratified.split(X, y):
    stratified_train = data.loc[train_set]
    stratified_test = data.loc[test_set]
    
train_df = stratified_train
test_df = stratified_test


# Let's Shuffle the data
train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)


# Train set (Normal training dataset)
x_train = train_df.drop('loan_status', axis=1)
y_train = train_df['loan_status']


# Test Dataset
x_test = test_df.drop('loan_status', axis=1)
y_test = test_df['loan_status']

del train_df
del test_df
del stratified_train
del stratified_test
gc.collect()
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)



In [ ]:
import lightgbm as lgb
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
param = {
    'objective': 'multiclass',
    'num_class':9,
    'metric': 'multi_logloss',
    'learning_rate':0.003,
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_leaves':150
}
num_round=100
gbm = lgb.train(param,
                lgb_train,
                num_boost_round=num_round,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)


In [ ]:
from sklearn.metrics import  accuracy_score
from mlxtend.evaluate import confusion_matrix
y_pred_lgm = []
for i in y_pred:
    y_pred_lgm.append(np.argmax(i))
#print(len(y_pred_lgm))
cm = confusion_matrix(y_test, y_pred_lgm,binary=True)
tp, fn, fp, tn = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
accuracy_lgbm = accuracy_score(y_pred_lgm,y_test)
print("LGBM",accuracy_lgbm)
print(tp, fn, fp, tn)
#recal = recall_score(y_pred_lgm,y_test)



In [ ]:
import gc
gc.collect()

In [ ]:
from sklearn import metrics
print(cm)
print(tp,fn,fp,tn)
recal = tp/(tp+fn)
precision = tp/(tp+fp)
acc = (tp + tn)/(tp+tn+fn+fp)
print(recal, precision, acc)


In [ ]:
import xgboost as xgb 
dtrain = xgb.DMatrix(x_train,label=y_train)
dtest=xgb.DMatrix(x_test)
param = {
    'objective': 'multi:softmax',
    'num_class':9,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_leaves':150
}
num_round=50
xg=xgb.train(param,dtrain,num_round)
ypred=xg.predict(dtest)
cm = confusion_matrix(y_test, ypred,binary=True)
print(cm)
accuracy_xgb = accuracy_score(y_test,ypred) 
print("XGBOOT:",accuracy_xgb)


In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=50,depth=7,learning_rate=0.05,loss_function='MultiClass')
model.fit(x_train, y_train)
pred_cat = model.predict(x_test)
cm = confusion_matrix(y_test, pred_cat,binary=True)
print(cm)
accuracy_cat = accuracy_score(pred_cat,y_test)
print("Catboost",accuracy_cat)

In [ ]:
from scipy import stats
from sklearn.metrics import accuracy_score
stack = []
for i in range(len(ypred)):
    if ypred[i] == y_pred_lgm[i] == pred_cat[i]:
        stack.append(int(ypred[i]))
    elif int(stats.mode([ypred[i],y_pred_lgm[i],pred_cat[i]]).count) == 2:
        stack.append(int(stats.mode([ypred[i],y_pred_lgm[i],pred_cat[i]]).mode))
    else:
        stack.append(int(ypred[i]))
cm = confusion_matrix(y_test, stack)
print(cm)
accuracy_stack = accuracy_score(y_test,np.asarray(stack))
print("Stacking:",accuracy_stack)

In [ ]:
pred_cat = pred_cat.ravel()